In [5]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.0 MB/s eta 0:00:00


In [96]:
import pandas as pd
from datasets import load_dataset
from kaggle_secrets import UserSecretsClient
from openai import OpenAI
import openai

import re
from typing import List
import ast
from tqdm import tqdm

In [10]:
def hard_processing(text):
    text = re.sub(r"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d', '', text)
    text = re.sub(r'\b\w\b\s?', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()

def comp_metric(y_true: List[List[str]], y_pred: List[List[str]]):
    assert len(y_true) == len(y_pred)
    tp, fp, fn, p,  = 0.0, 0.0, 0.0, 0.0

    for y_true_sample, y_pred_sample in zip(y_true, y_pred):
        y_true_sample = set([hard_processing(s) for s in y_true_sample])
        y_pred_sample = set([hard_processing(s) for s in y_pred_sample])

        tp += len(y_true_sample & y_pred_sample)
        fp += len(y_pred_sample - y_true_sample)
        fn += len(y_true_sample - y_pred_sample)
        p += len(y_true_sample)

    if tp + fp == 0:
        if p == 0:
            precision = 1.0
        else:
            precision = 0.0
    else:
        precision = tp/(tp + fp)

    if tp + fn == 0:
        if p == 0:
            recall = 1.0
        else:
            recall = 0.0
    else:
        recall = tp/(tp + fn)

    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [93]:
API_KEY = UserSecretsClient().get_secret("openai-api-key")
client = OpenAI(
  api_key=API_KEY,
)

In [101]:
NER_PROMPT = """
Your are experienced in cities and locations of Ukraine.
Your task is to perform NER and extract Ukrainian locations from either Ukrainian or Russian text.
You should return python list with extracted locations.

Few-shots:
Зокрема, у квітні 2014 року на брифингу про завершення перевірки відносно законності передачі в оренду землі під "Межигір'ям".
["Межигір'ям"]

Рух не має формального лідера, та єдиної форми організації.
[]

ПСЖ обіграв Нант, перемоги Монако та Сент-Етьєна.
[]

Кореспонденти ТСН з Києва, Миколаєва та Рівного перевірили, як триває підготовка до виборів під час пандемії.
['Києва', 'Миколаєва', 'Рівного']

Загреб прагнув повернути в європейський порядок денний питання розширення ЄС за рахунок Балканських держав.
['Загреб', 'Балканських']

Володимир Зеленський каже, що Кремль хоче повернути Радянський союз, але така політика помилкова.
['Кремль', 'Радянський']

INPUT TEXT:
<text>
"""
def generate_response(text: str, max_tokens: int=1000):
    if not openai.api_key:
        raise ValueError("OpenAI API key is not set.")
        
    prompt = NER_PROMPT.replace('<text>', text)

    response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.7
    )

    generated_text = response.choices[0].text.strip()
    return generated_text

text = "Чи підтримуєш ти створення на Донбасі вільної економічної зони?"
generate_response(text)

"['Донбасі']"

In [109]:
df = pd.read_csv('/kaggle/input/val-df/val_df.csv')[['text', 'conll_tokens', 'ner_tags']].sample(1000)
df.shape

(1000, 3)

In [110]:
def extract_locs_with_ner_tag(text, ner_tags):
    ner_tags = ast.literal_eval(ner_tags)
    text = ast.literal_eval(text)
    res = [token for token, ner_tag in zip(text, ner_tags) if ner_tag == 1]
    return res

df['locations'] = df.apply(lambda row: extract_locs_with_ner_tag(row['conll_tokens'], row['ner_tags']), axis=1)
df.sample(5)

,text,conll_tokens,ner_tags,locations
6996,У Зеленського анонсували “повний перезапуск” с...,"['У', 'Зеленського', 'анонсували', '“', 'повни...","[0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, ...",[]
23916,"Примітно, що спершу рокер написав пісню для ам...","['Примітно', ',', 'що', 'спершу', 'рокер', 'на...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
16991,Окрім постійного дефіциту найнеобхіднішого це ...,"['Окрім', 'постійного', 'дефіциту', 'найнеобхі...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[]
4632,"Якщо всі знають, що хабарі дають в доларах і м...","['Якщо', 'всі', 'знають', ',', 'що', 'хабарі',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[]
32570,Якщо мета цього зображення просто статеве збуд...,"['Якщо', 'мета', 'цього', 'зображення', 'прост...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[]


In [111]:
prompt_shots = ""
for i, row in df.iterrows():
    text = row['text']
    locations = row['locations']
    
    if 0 < len(locations) < 10:
        prompt_shots += f"{text}\n{locations}\n\n"
    else:
        break

In [112]:
llm_locations = []
real_locations = []

with tqdm(total=df.shape[0]) as pbar: 
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        pbar.update(1)
        text = row['text']
        locations = row['locations']

        real_locations.append(locations)
        extracted_locations = generate_response(text)
        if len(extracted_locations) > 4:
            try:
                extracted_locations = ast.literal_eval(extracted_locations)
            except:
                extracted_locations = []
        else:
            extracted_locations = []
        llm_locations.append(extracted_locations)
pbar.close()


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 2/1000 [00:00<02:33,  6.52it/s]

  0%|          | 3/1000 [00:00<04:52,  3.40it/s]

  0%|          | 4/1000 [00:01<05:24,  3.07it/s]

  0%|          | 5/1000 [00:01<06:07,  2.70it/s]

  1%|          | 6/1000 [00:02<06:21,  2.61it/s]

  1%|          | 7/1000 [00:02<06:39,  2.49it/s]

  1%|          | 8/1000 [00:02<06:29,  2.55it/s]

  1%|          | 9/1000 [00:03<06:01,  2.74it/s]

  1%|          | 10/1000 [00:03<07:24,  2.23it/s]

  1%|          | 11/1000 [00:04<07:05,  2.32it/s]

  1%|          | 12/1000 [00:04<08:02,  2.05it/s]

  1%|▏         | 13/1000 [00:05<07:29,  2.20it/s]

  1%|▏         | 14/1000 [00:07<14:25,  1.14it/s]

  2%|▏         | 15/1000 [00:07<11:24,  1.44it/s]

  2%|▏         | 16/1000 [00:07<10:38,  1.54it/s]

  2%|▏         | 17/1000 [00:08<09:17,  1.76it/s]

  2%|▏         | 18/1000 [00:08<08:20,  1.96it/s]

  2%|▏         | 19/1000 [00:08<07:32,  2.17it/s]

  2%|

In [114]:
comp_metric(llm_locations, real_locations)

{'precision': 0.8176470588235294,
 'recall': 0.24514991181657847,
 'f1': 0.37720488466757124}

In [ ]:
df_test[['text_id', 'locations']].to_csv('spacy_baseline.csv', index=False)